In [1]:
## Connect to DB server on AWS
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()

AttributeError: module 'config' has no attribute 'pw'

In [2]:
# ## Create new DB 
# cursor.execute("CREATE DATABASE yelp")

In [ ]:
# connect to database
cnx.database = 'yelp'

In [ ]:
# # Create a table for the Businesses
# create_table_business = """
#     CREATE TABLE businesses (
#     businessID varchar(255) PRIMARY KEY,
#     name varchar(255),
#     rating FLOAT,
#     review_count INTEGER,
#     price varchar(255)
#     );

# """

In [ ]:
# cursor.execute(create_table_business)

In [ ]:
# create_table_review = """
#     CREATE TABLE reviews (
#     reviewID varchar(255) PRIMARY KEY,
#     text_of_review varchar(255),
#     businessID varchar(255),
#     FOREIGN KEY (businessID)
#         REFERENCES businesses(businessID)
#     );
# """

In [ ]:
# cursor.execute(create_table_review)

In [ ]:
import requests
import json

In [ ]:
# Write a function to make a call to the yelp API
business_url = 'https://api.yelp.com/v3/businesses/search'

In [ ]:
client_id = '2uBXLKdIeFK2UNqhwXOPRQ'
api_key = 'x_KJc7Xc556wdB3hXmC82fjHZ4hKR_KUzdaarXmcqnvkIi_7xJDme-Q4ze0TDTsY4BMag0PbgElDq2ax6Yn0u16KgnvWtVrrMEMjkRndEJZoDnURYccdtH4QNjTpXXYx'

In [ ]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [ ]:
# what type of business do you want to search
term = 'Spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against
status = True

In [ ]:
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status
            }


In [ ]:
response = requests.get(business_url, headers=headers, params=url_params)

In [ ]:
business_data = response.json()
business_data

In [ ]:
business_url = 'https://api.yelp.com/v3/businesses/search'
client_id = '2uBXLKdIeFK2UNqhwXOPRQ'
api_key = 'x_KJc7Xc556wdB3hXmC82fjHZ4hKR_KUzdaarXmcqnvkIi_7xJDme-Q4ze0TDTsY4BMag0PbgElDq2ax6Yn0u16KgnvWtVrrMEMjkRndEJZoDnURYccdtH4QNjTpXXYx'


headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


# what type of business do you want to search
term = 'Spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against
status = True


url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status
            }


def yelp_call(url_params, api_key):
    response = requests.get(business_url, headers=headers, params=url_params)
    business_data = response.json()
    return business_data

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

def parse_data(data):
    
    # create a container to hold our parsed data
    
    parsed_data = []
    
    # loop through our business and parse each individual business
    # in the loop, add each individual parsed business to our data container
    # return the container with all of the parsed results
    
    for business in data['businesses']:
        if 'price' not in business.keys():
            business.setdefault('price', None)
        biz_tuple = (business['id'], business['name'], business['rating'], business['review_count'], business['price'])
        parsed_data.append(biz_tuple)
    return parsed_data

parse_data(business_data)


In [ ]:
business_data['businesses'][0].keys()

In [ ]:
# Write a function to take your parsed data and insert it into the DB

insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""

In [ ]:
cursor.executemany(insert_business, parsed_businesses)

In [ ]:
cnx.commit()

In [ ]:
cursor.execute("""DELETE FROM businesses;""")
cnx.commit()

In [ ]:
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()
cnx.database = 'yelp'

In [ ]:
def insert_data(cnx, cursor, parsed_data):
    insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""
    cursor.executemany(insert_business, parsed_data)
    cnx.commit()
    return

In [ ]:
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 

cur = 21

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    business_data = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_data = parse_data(business_data)
    
    # use your function to insert your parsed results into the db
    insert_data(cnx, cursor, parsed_data)
    #increment the counter by 50 to move on to the next results
    cur += 20